In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import gensim
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from tqdm import tqdm
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from nltk import word_tokenize
import re
import concurrent.futures
import time
import pickle
from collections import defaultdict

import string

In [2]:
train_data_raw = pd.read_csv('/home/ubuntu/train.csv')
test_data_raw = pd.read_csv('/home/ubuntu/test.csv')
train_data_raw.to_pickle('train_data_raw.pkl')
test_data_raw.to_pickle('test_data_raw.pkl')

/home/ubuntu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
abbr_dict={
"ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have" ,
                   "date of birth" : "DOB" , "birth date" : "DOB" ,"birthdate" : "DOB",

}

def another_clean(s):
    # Abreviations
    s=s.replace('Assn.', 'Association')
    s=s.replace('Mt.', 'Mount')
    s=s.replace('Ave.', 'Avenue')
    s=s.replace('St.', 'Street')
    s=s.replace('Dept.', 'Department')
    s=s.replace('No.', 'Number')
    s=s.replace(' no.', ' Number')
    s=s.replace('etc.', ' ')
    s=s.replace(' vs ',' versus ')
    s=s.replace(' vs. ',' versus ')
    s=s.replace('btwn','between')
    s=s.replace('Btwn','Between')
    s=s.replace(' u ',' you ')
    
    #Noms propres, acronymes
    s=re.sub(r'(the USA)(?=[^a-zA-Z])',"America", s)
    s=re.sub(r'(the U.S.A.)(?=[^a-zA-Z])',"America", s)
    s=re.sub(r'(The U.S.A)(?=[^a-zA-Z])',"America", s)
    s=re.sub(r'(The U.S)(?=[^a-zA-Z])',"America", s)
    s=re.sub(r'(the US)(?=[^a-zA-Z])',"America", s)
    s=re.sub(r'(the U.S)(?=[^a-zA-Z])',"America", s)
    s=re.sub(r'(The U.S)(?=[^a-zA-Z])',"America", s)
    s=re.sub(r'(The U.S)(?=[^a-zA-Z])',"America", s)
    s=s.replace('United States of America', 'America')
    s=s.replace('United States', 'America')
    s=s.replace('World Trade Organization', 'WTO')
    s=re.sub(r'( UK)(?=[^a-zA-Z])'," United Kingdom", s)
    s=re.sub(r'( uk)(?=[^a-zA-Z])'," United Kingdom", s)
    s=s.replace('date of birth', 'DOB')
    s=s.replace('birth date','DOB')
    s=s.replace('birthdate','DOB')
    s=s.replace('M.Sc.','MSc')
    s=s.replace('B.Sc.','BSc')
    s=s.replace('B. Tech.','BTech')
    s=s.replace('Master of Science','MSc')
    s=s.replace('Bachelor of Science','BSc')
    s=s.replace('Bachelor of Technology','BTech')
    s=s.replace('Master of science','MSc')
    s=s.replace('Bachelor of science','BSc')
    s=s.replace('Bachelor of technology','BTech')
    s=s.replace('Ph.D.','PhD')
    s=s.replace('Ph.D','PhD')
    s=s.replace('doctorate','PhD')
    s=s.replace('New York','New-York')
    s=s.replace(' NY ',' New-York ')
    s=s.replace('Los Angeles','Los-Angeles')
    s=s.replace(' LA ',' Los-Angeles ')
    s=s.replace('San Francisco','San-Francisco')
    s=s.replace('United Nations','UN')
    s=s.replace('UPSC','civil service')
    s=s.replace('IAS','civil service')
    s=s.replace('upsc','civil service')
    s=s.replace('ias','civil service')
    s=s.replace('Orange is the New Black','OITNB')
    s=s.replace('World War III','WW3')
    s=s.replace('World War 3','WW3')
    s=s.replace('WWIII','WW3')
    s=s.replace('World War II','WW2')
    s=s.replace('World War 2','WW2')
    s=s.replace('WWII','WW2')
    s=s.replace('World War I','WW1')
    s=s.replace('World War 1','WW1')
    s=s.replace('WWI','WW1')
    s=s.replace('Third World War','WW3')
    s=s.replace('third world war','WW3')
    s=s.replace('Third world war','WW3')
    s=s.replace('Second World War','WW2')
    s=s.replace('second world war','WW2')
    s=s.replace('Second world war','WW2')
    s=s.replace('World War I','WW1')
    s=s.replace('World War 1','WW1')
    s=s.replace('WWI','WW1')
    s=s.replace('1st',"first")
    s=s.replace('2nd',"second")
    s=s.replace('3rd',"third")
    s=s.replace('4th',"fourth")
    s=s.replace('5th',"fifth")
    s=s.replace('6th',"sixth")
    s=s.replace('7th',"seventh")
    s=s.replace('8th',"eighth")
    s=s.replace('9th',"nineth")
    s=s.replace('Wat ', 'What')
    s=s.replace("(^|\W)\d+($|\W)", " ")
    return s

table = str.maketrans(dict.fromkeys(string.punctuation))


def preprocessing(question):
    #print question
    question=str(question).lower()
    for item in abbr_dict.items():
        question=question.replace(item[0],item[1])
        question = another_clean(question)
    #print question
    question=question.translate(table)
    #question=re.sub(r'[^\x00-\xff]+','non-english',question.decode('utf-8'))
    return str(question)


In [4]:
train_data_raw.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
test_data_raw.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [6]:
#对train.csv中的语句进行预处理
start = time.time()

with concurrent.futures.ProcessPoolExecutor() as executor:
    text_train_q1 = executor.map(preprocessing,train_data_raw['question1'].tolist())
    text_train_q2 = executor.map(preprocessing,train_data_raw['question2'].tolist())
    text_train_q1_lst = list(text_train_q1)
    text_train_q2_lst = list(text_train_q2)
    

temp = pd.DataFrame({'q1':text_train_q1_lst,'q2':text_train_q2_lst})
train_data = pd.DataFrame()
train_data['train_id'] = train_data_raw['id']
train_data = pd.concat([train_data,temp],axis=1)


elapsed = (time.time() - start)
print("Time used:",elapsed)

Time used: 321.60658621788025


In [7]:
train_data.head()

,train_id,q1,q2
0,0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...
1,1,what is the story of kohinoor kohinoor diamond,what would happen if the indian government sto...
2,2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...
3,3,why am i mentally very lonely how can i solve it,find the remainder when math2324math is divide...
4,4,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water


In [8]:
train_data.to_pickle('train_data_processed.pkl')

# basic feature

In [9]:
start = time.time()
train_data_features_basic = pd.DataFrame()
train_data_features_basic['len_q1'] = train_data['q1'].apply(lambda x:len(x))
train_data_features_basic['len_q2'] = train_data['q2'].apply(lambda x:len(x))
train_data_features_basic['diff_len'] = train_data_features_basic['len_q1'] - train_data_features_basic['len_q2']
train_data_features_basic['len_char_q1'] = train_data.q1.apply(lambda x:len(set(x.replace(' ',''))))
train_data_features_basic['len_char_q2'] = train_data.q2.apply(lambda x:len(set(x.replace(' ',''))))
train_data_features_basic['len_word_q1'] = train_data.q1.apply(lambda x:len(x.split()))
train_data_features_basic['len_word_q2'] = train_data.q2.apply(lambda x:len(x.split()))
train_data_features_basic['common_word'] = train_data.apply(lambda x: 
        len(set(x['q1'].split()) & set(x['q2'].translate(table).lower().split())),axis = 1)
stop_words = stopwords.words('english')
def word_match_share(row):
    q1words = {}
    q2words = {}
    for word in row['q1'].split():
        if word not in stop_words:
            q1words[word] = 1
    for word in row['q2'].split():
        if word not in stop_words:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0
    shared_words_in_q1 = [w for w in q1words.keys() if w in q2words]
    shared_words_in_q2 = [w for w in q2words.keys() if w in q1words]
    R = (len(shared_words_in_q1) + len(shared_words_in_q2))*1.0/(len(q1words) + len(q2words))
    return R

train_data_features_basic['R']=train_data.apply(lambda x : word_match_share(x), axis=1)
train_data_features_basic.to_pickle('train_data_features_basic.pkl')

elapsed = (time.time() - start)
print("Time used:",elapsed)


Time used: 45.16831350326538


# fuzzywuzzy

In [19]:
start = time.time()
train_data_features_fw = pd.DataFrame()
train_data_features_fw['simple_ratio'] = train_data.apply(lambda x:fuzz.ratio(x['q1'],x['q2']),axis = 1)
train_data_features_fw['partial_ratio'] = train_data.apply(lambda x:fuzz.partial_ratio(x['q1'],x['q2']),axis = 1)
train_data_features_fw['token_sort_ratio'] = train_data.apply(lambda x:fuzz.token_sort_ratio(x['q1'],x['q2']),axis = 1)
train_data_features_fw['partial_token_sort_ratio'] = train_data.apply(lambda x:fuzz.partial_token_sort_ratio(x['q1'],x['q2']),axis = 1)
train_data_features_fw['token_set_ratio'] = train_data.apply(lambda x:fuzz.token_set_ratio(x['q1'],x['q2']),axis = 1)
train_data_features_fw['partial_token_set_ratio'] = train_data.apply(lambda x:fuzz.partial_token_set_ratio(x['q1'],x['q2']),axis = 1)
train_data_features_fw['QRatio'] = train_data.apply(lambda x:fuzz.QRatio(x['q1'],x['q2']),axis = 1)
train_data_features_fw['UQRatio'] = train_data.apply(lambda x:fuzz.UQRatio(x['q1'],x['q2']),axis = 1)
train_data_features_fw['WRatio'] = train_data.apply(lambda x:fuzz.WRatio(x['q1'],x['q2']),axis = 1)
train_data_features_fw['UWRatio'] = train_data.apply(lambda x:fuzz.UWRatio(x['q1'],x['q2']),axis = 1)
train_data_features_fw.to_pickle('train_data_features_fw.pkl')
elapsed = (time.time() - start)
print("Time used:",elapsed)


Time used: 453.29890727996826


# Word2Vec

In [3]:
train_data = pd.read_pickle('train_data_processed.pkl')


In [4]:
train_data.head()

,train_id,q1,q2
0,0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...
1,1,what is the story of kohinoor kohinoor diamond,what would happen if the indian government sto...
2,2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...
3,3,why am i mentally very lonely how can i solve it,find the remainder when math2324math is divide...
4,4,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water


In [14]:
start = time.time()
stop_words = stopwords.words('english')
def wmd(s1, s2):
    s1 = str(s1).split()
    s2 = str(s2).split()
    stop_words = stopwords.words('english')
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return model.wmdistance(s1, s2)


def norm_wmd(s1, s2):
    s1 = str(s1).split()
    s2 = str(s2).split()
    stop_words = stopwords.words('english')
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return norm_model.wmdistance(s1, s2)


def sent2vec(s):
    words = str(s)#.decode('utf-8')
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v / np.sqrt((v ** 2).sum())


In [16]:
start = time.time()
train_data_features_w2v_1 = pd.DataFrame()
model = gensim.models.KeyedVectors.load_word2vec_format('/home/ubuntu/GoogleNews-vectors-negative300.bin.gz', 
                                                        binary=True)
train_data_features_w2v_1['wmd'] = train_data.apply(lambda x: wmd(x['q1'], x['q2']), axis=1)
norm_model = gensim.models.KeyedVectors.load_word2vec_format('/home/ubuntu/GoogleNews-vectors-negative300.bin.gz', 
                                                        binary=True)
norm_model.init_sims(replace=True)
train_data_features_w2v_1['norm_wmd'] = train_data.apply(lambda x: norm_wmd(x['q1'], x['q2']), axis=1)

train_data_features_w2v_1.to_pickle('train_data_features_w2v_1.pkl')
elapsed = (time.time() - start)
print("Time used:",elapsed)

Time used: 998.3847367763519


In [17]:
start = time.time()
train_q1_vectors = np.zeros((train_data.shape[0], 300))
error_count = 0

# with concurrent.futures.ProcessPoolExecutor() as executor:
#     aaa = executor.map(sent2vec,train_data.q1.values)
#     train_q1_vectors = np.array(list(aaa))
#     bbb = executor.map(sent2vec,train_data.q2.values)
#     train_q2_vectors = np.array(list(bbb))
    

             
             
             
for i, q in tqdm(enumerate(train_data.q1.values)):
    train_q1_vectors[i, :] = sent2vec(q)

train_q2_vectors  = np.zeros((train_data.shape[0], 300))
for i, q in tqdm(enumerate(train_data.q2.values)):
    train_q2_vectors[i, :] = sent2vec(q)
    
# with open('train_q1_vectors.pkl','wb') as file_1:
#     np.save(file_1, train_q1_vectors)
    
# with open('train_q2_vectors.pkl','wb') as file_2:
#     np.save(file_2, train_q2_vectors)
    

elapsed = (time.time() - start)
print("Time used:",elapsed)

0it [00:00, ?it/s]/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in double_scalars
404290it [01:26, 4688.48it/s]
404290it [01:26, 4685.34it/s]

Time used: 172.52987217903137


In [18]:
start = time.time()
train_data_features_w2v_2 = pd.DataFrame()
train_data_features_w2v_2['cosine_distance'] = [cosine(x, y) for (x, y) in zip(np.nan_to_num(train_q1_vectors),
                                                          np.nan_to_num(train_q2_vectors))]


train_data_features_w2v_2['cityblock_distance'] = [cityblock(x, y) for (x, y) in zip(np.nan_to_num(train_q1_vectors),
                                                          np.nan_to_num(train_q2_vectors))]

train_data_features_w2v_2['jaccard_distance'] = [jaccard(x, y) for (x, y) in zip(np.nan_to_num(train_q1_vectors),
                                                          np.nan_to_num(train_q2_vectors))]

train_data_features_w2v_2['canberra_distance'] = [canberra(x, y) for (x, y) in zip(np.nan_to_num(train_q1_vectors),
                                                          np.nan_to_num(train_q2_vectors))]

train_data_features_w2v_2['euclidean_distance'] = [euclidean(x, y) for (x, y) in zip(np.nan_to_num(train_q1_vectors),
                                                          np.nan_to_num(train_q2_vectors))]

train_data_features_w2v_2['minkowski_distance'] = [minkowski(x, y, 3) for (x, y) in zip(np.nan_to_num(train_q1_vectors),
                                                          np.nan_to_num(train_q2_vectors))]

train_data_features_w2v_2['braycurtis_distance'] = [braycurtis(x, y) for (x, y) in zip(np.nan_to_num(train_q1_vectors),
                                                          np.nan_to_num(train_q2_vectors))]

train_data_features_w2v_2['skew_q1vec'] = [skew(x) for x in np.nan_to_num(train_q1_vectors)]
train_data_features_w2v_2['skew_q2vec'] = [skew(x) for x in np.nan_to_num(train_q2_vectors)]
train_data_features_w2v_2['kur_q1vec'] = [kurtosis(x) for x in np.nan_to_num(train_q1_vectors)]
train_data_features_w2v_2['kur_q2vec'] = [kurtosis(x) for x in np.nan_to_num(train_q2_vectors)]
train_data_features_w2v_2.to_pickle('train_data_features_w2v_2.pkl')
elapsed = (time.time() - start)
print("Time used:",elapsed)


/home/ubuntu/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/home/ubuntu/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:853: RuntimeWarning: invalid value encountered in double_scalars
  dist = np.double(unequal_nonzero.sum()) / np.double(nonzero.sum())
/home/ubuntu/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:1138: RuntimeWarning: invalid value encountered in double_scalars
  return l1_diff.sum() / l1_sum.sum()


Time used: 422.0957283973694


# 处理test data

In [20]:
test_data_raw.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [21]:
start_all = time.time()
total = len(test_data_raw.test_id)
num = 30
div = total//num
for ii in range(num):
    print(ii)
    start = time.time()

    with concurrent.futures.ProcessPoolExecutor() as executor:
        
        text_test_q1 = list(executor.map(preprocessing,test_data_raw['question1'][ii*div:(ii+1)*div].tolist()))
        text_test_q2 = list(executor.map(preprocessing,test_data_raw['question2'][ii*div:(ii+1)*div].tolist()))

        pd.DataFrame({'q1':text_test_q1,'q2':text_test_q2}).to_pickle('test_data_processed_'+str(ii)+'.pkl')
    elapsed = (time.time() - start)
    print("Time used:",elapsed)
        
with concurrent.futures.ProcessPoolExecutor() as executor:
    text_test_q1 = list(executor.map(preprocessing,test_data_raw['question1'][num*div:].tolist()))
    text_test_q2 = list(executor.map(preprocessing,test_data_raw['question2'][num*div:].tolist()))

pd.DataFrame({'q1':text_test_q1,'q2':text_test_q2}).to_pickle('test_data_processed_'+str(num)+'.pkl')
elapsed = (time.time() - start_all)
print("Time used:",elapsed)

0
Time used: 95.53794431686401
1
Time used: 96.30256247520447
2
Time used: 96.33521842956543
3
Time used: 96.08878135681152
4
Time used: 96.5361340045929
5
Time used: 96.56264090538025
6
Time used: 96.62726998329163
7
Time used: 96.38904643058777
8
Time used: 96.17015218734741
9
Time used: 95.47581839561462
10
Time used: 96.31942701339722
11
Time used: 97.18250703811646
12
Time used: 96.36827039718628
13
Time used: 96.32027339935303
14
Time used: 95.45679807662964
15
Time used: 96.37856388092041
16
Time used: 97.37309384346008
17
Time used: 96.26779580116272
18
Time used: 96.38499569892883
19
Time used: 95.27958846092224
20
Time used: 96.31280779838562
21
Time used: 96.4473614692688
22
Time used: 97.26343107223511
23
Time used: 96.60183596611023
24
Time used: 95.44789576530457
25
Time used: 96.29660320281982
26
Time used: 97.52471542358398
27
Time used: 96.28169703483582
28
Time used: 95.4722671508789
29
Time used: 96.24962830543518
Time used: 2889.94522023201


In [2]:
train_data = pd.read_pickle('train_data_processed.pkl')
test_data = pd.DataFrame()
for ii in range(31):
    aaa = pd.read_pickle('test_data_processed_'+str(ii)+'.pkl')
    test_data = pd.concat([test_data,aaa],axis = 0)


In [3]:
train_data.head()

,train_id,q1,q2
0,0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...
1,1,what is the story of kohinoor kohinoor diamond,what would happen if the indian government sto...
2,2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...
3,3,why am i mentally very lonely how can i solve it,find the remainder when math2324math is divide...
4,4,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water


In [4]:
test_data.head()

,q1,q2
0,how does the surface pro himself 4 compare wit...,why did microsoft choose core m3 and not core ...
1,should i have a hair transplant at age 24 how ...,how much cost does hair transplant require
2,what but is the best way to send money from ch...,what you send money to china
3,which food not emulsifiers,what foods fibre
4,how aberystwyth start reading,how their can i start reading


In [5]:
test_data.

(3563475, 2)

In [7]:
df1 = train_data[['q1']].copy()
df2 = train_data[['q2']].copy()
df1_test = test_data[['q1']].copy()
df2_test = test_data[['q2']].copy()

df2.rename(columns = {'q2':'q1'},inplace=True)
df2_test.rename(columns = {'q2':'q1'},inplace=True)

train_questions = df1.append(df2)
train_questions = train_questions.append(df1_test)
train_questions = train_questions.append(df2_test)
#train_questions.drop_duplicates(subset = ['qid1'],inplace=True)
train_questions.drop_duplicates(subset = ['q1'],inplace=True)

train_questions.reset_index(inplace=True,drop=True)
questions_dict = pd.Series(train_questions.index.values,index=train_questions.q1.values).to_dict()
train_cp = train_data.copy()
test_cp = test_data.copy()
train_cp.drop(['train_id'],axis=1,inplace=True)

train_cp['is_duplicate'] = 1
test_cp['is_duplicate'] = -1
#test_cp.rename(columns={'test_id':'id'},inplace=True)
comb = pd.concat([train_cp,test_cp])

comb['q1_hash'] = comb['q1'].map(questions_dict)
comb['q2_hash'] = comb['q2'].map(questions_dict)

q1_vc = comb.q1_hash.value_counts().to_dict()
q2_vc = comb.q2_hash.value_counts().to_dict()

def try_apply_dict(x,dict_to_apply):
    try:
        return dict_to_apply[x]
    except KeyError:
        return 0
#map to frequency space
comb['q1_freq'] = comb['q1_hash'].map(lambda x: try_apply_dict(x,q1_vc) + try_apply_dict(x,q2_vc))
comb['q2_freq'] = comb['q2_hash'].map(lambda x: try_apply_dict(x,q1_vc) + try_apply_dict(x,q2_vc))

train_comb = comb[comb['is_duplicate'] >= 0][['q1_hash','q2_hash','q1_freq','q2_freq']]
test_comb = comb[comb['is_duplicate'] < 0][['q1_hash','q2_hash','q1_freq','q2_freq']]

train_comb.to_pickle('train_data_features_magic_v1.pkl')
test_comb.to_pickle('test_data_features_magic_v1.pkl')

In [ ]:
ques = pd.concat([train_data[['q1', 'q2']], \
        test_data[['q1', 'q2']]], axis=0).reset_index(drop='index')
q_dict = defaultdict(set)
for i in range(ques.shape[0]):
        q_dict[ques.q1[i]].add(ques.q2[i])
        q_dict[ques.q2[i]].add(ques.q1[i])
        
def q1_q2_intersect(row):
    return(len(set(q_dict[row['q1']]).intersection(set(q_dict[row['q2']]))))

train_features_magic_v2 = pd.DataFrame()
test_features_magic_v2 = pd.DataFrame()
train_features_magic_v2['q1_q2_intersect'] = train_data.apply(lambda x:q1_q2_intersect(x), axis=1)
test_features_magic_v2['q1_q2_intersect'] = test_data.apply(lambda x:q1_q2_intersect(x), axis=1)

train_features_magic_v2.to_pickle('train_data_features_magic_v2.pkl')
test_features_magic_v2.to_pickle('test_data_features_magic_v2.pkl')

In [16]:
train_features_magic_v2['q1_q2_intersect'] = train_data.apply(lambda x:q1_q2_intersect(x), axis=1)

In [18]:
train_features_magic_v2.head(10)

,q1_q2_intersect
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,1
9,0


In [19]:

test_features_magic_v2['q1_q2_intersect'] = test_data.apply(lambda x:q1_q2_intersect(x), axis=1)

train_features_magic_v2.to_pickle('train_data_features_magic_v2.pkl')
test_features_magic_v2.to_pickle('test_data_features_magic_v2.pkl')

In [2]:
stop_words = stopwords.words('english')
def word_match_share(row):
    q1words = {}
    q2words = {}
    for word in row['q1'].split():
        if word not in stop_words:
            q1words[word] = 1
    for word in row['q2'].split():
        if word not in stop_words:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0
    shared_words_in_q1 = [w for w in q1words.keys() if w in q2words]
    shared_words_in_q2 = [w for w in q2words.keys() if w in q1words]
    R = (len(shared_words_in_q1) + len(shared_words_in_q2))*1.0/(len(q1words) + len(q2words))
    return R


def wmd(s1, s2):
    s1 = str(s1).split()
    s2 = str(s2).split()
    stop_words = stopwords.words('english')
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return model.wmdistance(s1, s2)


def norm_wmd(s1, s2):
    s1 = str(s1).split()
    s2 = str(s2).split()
    stop_words = stopwords.words('english')
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return norm_model.wmdistance(s1, s2)


def sent2vec(s):
    words = str(s)#.decode('utf-8')
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v / np.sqrt((v ** 2).sum())

model = gensim.models.KeyedVectors.load_word2vec_format('/home/ubuntu/GoogleNews-vectors-negative300.bin.gz', 
                                                        binary=True)
norm_model = gensim.models.KeyedVectors.load_word2vec_format('/home/ubuntu/GoogleNews-vectors-negative300.bin.gz', 
                                                        binary=True)
norm_model.init_sims(replace=True)
table = str.maketrans(dict.fromkeys(string.punctuation))

In [3]:
start_all = time.time()
for ii in range(31):
    print(ii)
    #test_features = pd.DataFrame()
    test_data = pd.read_pickle('test_data_processed_'+str(ii)+'.pkl')
    ## basic
    start = time.time()
    test_data_features_basic = pd.DataFrame()
    test_data_features_basic['len_q1'] = test_data['q1'].apply(lambda x:len(x))
    test_data_features_basic['len_q2'] = test_data['q2'].apply(lambda x:len(x))
    test_data_features_basic['diff_len'] = test_data_features_basic['len_q1'] - test_data_features_basic['len_q2']
    test_data_features_basic['len_char_q1'] = test_data.q1.apply(lambda x:len(set(x.replace(' ',''))))
    test_data_features_basic['len_char_q2'] = test_data.q2.apply(lambda x:len(set(x.replace(' ',''))))
    test_data_features_basic['len_word_q1'] = test_data.q1.apply(lambda x:len(x.split()))
    test_data_features_basic['len_word_q2'] = test_data.q2.apply(lambda x:len(x.split()))
    test_data_features_basic['common_word'] = test_data.apply(lambda x: 
            len(set(x['q1'].split()) & set(x['q2'].translate(table).lower().split())),axis = 1)
    test_data_features_basic['R']=test_data.apply(lambda x : word_match_share(x), axis=1)
    elapsed = (time.time() - start)
    print(str(ii)+"Time used(basic):",elapsed)
    
    ## fuzzywuzzy
    start = time.time()
    test_data_features_fw = pd.DataFrame()
    test_data_features_fw['simple_ratio'] = test_data.apply(lambda x:fuzz.ratio(x['q1'],x['q2']),axis = 1)
    test_data_features_fw['partial_ratio'] = test_data.apply(lambda x:fuzz.partial_ratio(x['q1'],x['q2']),axis = 1)
    test_data_features_fw['token_sort_ratio'] = test_data.apply(lambda x:fuzz.token_sort_ratio(x['q1'],x['q2']),axis = 1)
    test_data_features_fw['partial_token_sort_ratio'] = test_data.apply(lambda x:fuzz.partial_token_sort_ratio(x['q1'],x['q2']),axis = 1)
    test_data_features_fw['token_set_ratio'] = test_data.apply(lambda x:fuzz.token_set_ratio(x['q1'],x['q2']),axis = 1)
    test_data_features_fw['partial_token_set_ratio'] = test_data.apply(lambda x:fuzz.partial_token_set_ratio(x['q1'],x['q2']),axis = 1)
    test_data_features_fw['QRatio'] = test_data.apply(lambda x:fuzz.QRatio(x['q1'],x['q2']),axis = 1)
    test_data_features_fw['UQRatio'] = test_data.apply(lambda x:fuzz.UQRatio(x['q1'],x['q2']),axis = 1)
    test_data_features_fw['WRatio'] = test_data.apply(lambda x:fuzz.WRatio(x['q1'],x['q2']),axis = 1)
    test_data_features_fw['UWRatio'] = test_data.apply(lambda x:fuzz.UWRatio(x['q1'],x['q2']),axis = 1)
    elapsed = (time.time() - start)
    print(str(ii)+"Time used(fw):",elapsed)
    
    ## word2vec
    ## 1
    start = time.time()
    test_data_features_w2v_1 = pd.DataFrame()
    test_data_features_w2v_1['wmd'] = test_data.apply(lambda x: wmd(x['q1'], x['q2']), axis=1)
    test_data_features_w2v_1['norm_wmd'] = test_data.apply(lambda x: norm_wmd(x['q1'], x['q2']), axis=1)
    elapsed = (time.time() - start)
    print(str(ii)+"Time used(w2v_1):",elapsed)
    ## 2
    start = time.time()
    test_q1_vectors = np.zeros((test_data.shape[0], 300))            
    for i, q in tqdm(enumerate(test_data.q1.values)):
        test_q1_vectors[i, :] = sent2vec(q)

    test_q2_vectors  = np.zeros((test_data.shape[0], 300))
    for i, q in tqdm(enumerate(test_data.q2.values)):
        test_q2_vectors[i, :] = sent2vec(q)
    elapsed = (time.time() - start)
    print(str(ii)+"Time used(w2v_2):",elapsed)
    ## 3
    start = time.time()
    test_data_features_w2v_2 = pd.DataFrame()
    test_data_features_w2v_2['cosine_distance'] = [cosine(x, y) for (x, y) in zip(np.nan_to_num(test_q1_vectors),
                                                          np.nan_to_num(test_q2_vectors))]


    test_data_features_w2v_2['cityblock_distance'] = [cityblock(x, y) for (x, y) in zip(np.nan_to_num(test_q1_vectors),
                                                          np.nan_to_num(test_q2_vectors))]

    test_data_features_w2v_2['jaccard_distance'] = [jaccard(x, y) for (x, y) in zip(np.nan_to_num(test_q1_vectors),
                                                          np.nan_to_num(test_q2_vectors))]

    test_data_features_w2v_2['canberra_distance'] = [canberra(x, y) for (x, y) in zip(np.nan_to_num(test_q1_vectors),
                                                          np.nan_to_num(test_q2_vectors))]

    test_data_features_w2v_2['euclidean_distance'] = [euclidean(x, y) for (x, y) in zip(np.nan_to_num(test_q1_vectors),
                                                          np.nan_to_num(test_q2_vectors))]

    test_data_features_w2v_2['minkowski_distance'] = [minkowski(x, y, 3) for (x, y) in zip(np.nan_to_num(test_q1_vectors),
                                                          np.nan_to_num(test_q2_vectors))]

    test_data_features_w2v_2['braycurtis_distance'] = [braycurtis(x, y) for (x, y) in zip(np.nan_to_num(test_q1_vectors),
                                                          np.nan_to_num(test_q2_vectors))]

    test_data_features_w2v_2['skew_q1vec'] = [skew(x) for x in np.nan_to_num(test_q1_vectors)]
    test_data_features_w2v_2['skew_q2vec'] = [skew(x) for x in np.nan_to_num(test_q2_vectors)]
    test_data_features_w2v_2['kur_q1vec'] = [kurtosis(x) for x in np.nan_to_num(test_q1_vectors)]
    test_data_features_w2v_2['kur_q2vec'] = [kurtosis(x) for x in np.nan_to_num(test_q2_vectors)]
    elapsed = (time.time() - start)
    print(str(ii)+"Time used(w2v_3):",elapsed)
    
    test_features = pd.concat([test_data_features_basic,test_data_features_fw,test_data_features_w2v_1,test_data_features_w2v_2],axis = 1)
    test_features.to_pickle('test_features(no_magic)_'+str(ii)+'.pkl')

elapsed = (time.time() - start_all)
print("Time(all) used:",elapsed)

0
0Time used(basic): 12.809193849563599
0Time used(fw): 141.9659698009491


0it [00:00, ?it/s]/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in double_scalars
461it [00:00, 4602.40it/s]

0Time used(w2v_1): 222.02990746498108


118782it [00:24, 4832.49it/s]
118782it [00:24, 4819.12it/s]


0Time used(w2v_2): 49.23145842552185


...

29Time used(w2v_2): 49.09807586669922


/home/ubuntu/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/home/ubuntu/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:853: RuntimeWarning: invalid value encountered in double_scalars
  dist = np.double(unequal_nonzero.sum()) / np.double(nonzero.sum())
/home/ubuntu/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:1138: RuntimeWarning: invalid value encountered in double_scalars
  return l1_diff.sum() / l1_sum.sum()
15it [00:00, 3933.39it/s]

29Time used(w2v_3): 111.53040361404419
30
30Time used(basic): 0.008590221405029297
30Time used(fw): 0.044507741928100586
30Time used(w2v_1): 0.040991783142089844



15it [00:00, 4426.55it/s]

30Time used(w2v_2): 0.024122953414916992
30Time used(w2v_3): 0.020199060440063477
Time(all) used: 16157.25825047493
